### Use this for testing MONAI transforms

### Libraries and auxiliar functions

In [ ]:
import sys
import os
import json
import numpy as np
import random
import math
import itertools  

##########################################################
## VISUALIZATION

%matplotlib widget
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle, Circle, Arrow
from copy import copy

from ipywidgets import interact, interactive
from ipywidgets import widgets


def show_mag_phase_arrays(array_m, array_p, title=None, patch=None, margin=0.1, dpi=80, cmap="gray"):
    ysize = array_p.shape[1]
    xsize = array_p.shape[2]
    figsize = 2 *(1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi
    def callback(z=None):
        fig, ax = plt.subplots(1, 2, figsize=figsize, dpi=dpi)
        if z is None:
            ax[0].imshow(array_m,  interpolation=None, cmap=cmap)
            ax[1].imshow(array_p,  interpolation=None, cmap=cmap)
        else:
            ax[0].imshow(array_m[z, ...],  interpolation=None, cmap=cmap)
            ax[1].imshow(array_p[z, ...],  interpolation=None, cmap=cmap)
        ax[0].set_title('Magnitude')
        ax[1].set_title('Phase')
        if title:
            fig.suptitle(title)
        if patch:
            for element in patch:
                new_patch_1 = copy(element)
                new_patch_2 = copy(element)
                ax[0].add_patch(new_patch_1)
                ax[1].add_patch(new_patch_2)
        plt.show()
    interact(callback, z=(0, array_p.shape[0] - 1))

def show_complex_array(array_comp, title=None, patch=None, margin=0.1, dpi=80, cmap="gray"):
    (array_m, array_p) = get_mag_phase_arrays(array_comp)
    show_mag_phase_arrays(array_m, array_p, title, patch)

def show_array(array, title=None, patch=None, margin=0.1, dpi=80, cmap="gray"):
    ysize = array.shape[1]
    xsize = array.shape[2]
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi
    def callback(z=None):
        fig = plt.figure(figsize=figsize, dpi=dpi)
        ax = fig.add_axes([margin, margin, 1 - 2 * margin, 1 - 2 * margin])
        if z is None:
            ax.imshow(array,  interpolation=None, cmap=cmap)
        else:
            ax.imshow(array[z, ...],  interpolation=None, cmap=cmap)
        if title:
            plt.title(title)
        if patch:
            for element in patch:
                new_patch = copy(element)
                ax.add_patch(new_patch)
        plt.show()
        
    interact(callback, z=(0, array.shape[0] - 1))
##########################################################
## COMPLEX ARRAY MANIPULATION
# Return masked (non-complex) array
# output_pixel = value if mask_pixel == 0
# output_pixel = input_pixel if mask_pixel == 1
def array_mask(array, mask, value=0):
    foreground = np.multiply(array, mask)
    background = value*(np.ones_like(mask)-mask)
    masked_array = np.add(foreground, background)
    return masked_array
# Return masked complex array with scaling of phase (mean = 0)
def complex_diff_mask(array_c, mask):
    # Phase offset removal
    (array_m, array_p) = get_mag_phase_arrays(array_c)
    idx = (mask != 0)
    p_mean = np.mean(array_p[idx])    
    array_p = array_p - p_mean
    # Reconstruct foreground complex array
    array_c = get_complex_array(array_m, array_p)
    # Apply complex mask (mask with 1)
    return array_mask(array_c, mask, 1)
# From a complex array, get a pair of phase/mag arrays
def get_mag_phase_arrays(array_comp):
    array_m = np.absolute(array_comp)
    array_p = np.angle(array_comp)
    # Phase scaling
    p_max = np.max(array_p)
    p_min = np.min(array_p)
    if p_min < 0: # range [-pi, pi]
        array_p = array_p / p_max * np.pi
    else: # range [0, 2.*pi]
        array_p = array_p / p_max * 2.*np.pi
    return (array_m, array_p)
# From pair of mag/phase arrays, get complex array
def get_complex_array(array_m, array_p):
    # Scaling
    p_max = np.max(array_p)
    p_min = np.min(array_p)
    if p_min < 0: # range [-pi, pi]
        array_p = array_p / p_max * np.pi
    else: # range [0, 2.*pi]
        array_p = array_p / p_max * 2.*np.pi
    return (array_m * np.cos(array_p) + 1j * array_m * np.sin(array_p))

### Run code to be tested

In [ ]:
import glob
import os

import tempfile
import nibabel as nib
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable

import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, ArrayDataset, create_test_image_3d, DataLoader
from monai.transforms import (
    Transform,
    Compose,
    EnsureChannelFirst,
    LoadImage,
    Orientation,
    LoadImaged,
    Orientationd,
    EnsureChannelFirstd,
    ToTensord,
    Spacingd,
    CropForegroundd,
    RandCropByPosNegLabeld,
)

#### SELECT IMAGE TESTING SET
# base = 'TissueBase'     # AXIAL
base = 'PhantomBase'  # CORONAL

# Define orientation for rotation
axcode = 'RAS'
if base == 'TissueBase':
    axcode = 'SPL'
    pixel_dim = (2.0, 1.5625, 1.5625)
    window_size = (16, 192, 192)
elif base =='PhantomBase':
    axcode = 'ASL'
    pixel_dim = (5.0, 1.04167, 1.04167)
    window_size = (3, 192, 192)
    
    
srcdir = os.getcwd()

# Load images
print(os.path.join(srcdir, base+'_M'+'.nii.gz'))
images_m = sorted(glob.glob(os.path.join(srcdir, base+'_M'+'.nii.gz')))
images_p = sorted(glob.glob(os.path.join(srcdir, base+'_P'+'.nii.gz')))
labels = sorted(glob.glob(os.path.join(srcdir, base+'_label'+'.nii.gz')))

# Create dictionary
train_files = [
    {'image': image_name, 'label': label_name}
    for image_name, label_name in zip(images_m, labels)
]
img_file = train_files[0]['image']
label_file = train_files[0]['label']
fn_keys = ('image', 'label')  # filename keys for image and seg files

# Define transform
transfTest = Compose([LoadImage(image_only=True),
                EnsureChannelFirst(channel_dim='no_channel'), # This replaces deprecated AddChannel()
                Orientation(axcodes=axcode),
                ])

transf2 = Compose([LoadImaged(image_only=False, keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"], channel_dim='no_channel'), # Mariana: AddChanneld(keys=["image", "label"]) deprecated, use EnsureChannelFirst instead
                Orientationd(keys=["image", "label"], axcodes=axcode),
                Spacingd(keys=["image", "label"], pixdim=pixel_dim, mode=("bilinear", "nearest")),
                # CropForegroundd(keys=["image", "label"], source_key="image"),
                # RandCropByPosNegLabeld(
                #     keys=["image", "label"],
                #     label_key="label",
                #     spatial_size=window_size,
                #     pos=1,
                #     neg=1,
                #     num_samples=4,
                #     image_key="image",
                #     image_threshold=0,
                # ),
                # # user can also add other random transforms
                ToTensord(keys=["image", "label"]),    
    
])

img = transfTest(img_file)
img_arr = np.array(img)
print(np.shape(img_arr))
show_array(img_arr[0])

output_dict = transf2(train_files)
image_tf2 = output_dict[0]['image']
label_tf2 = output_dict[0]['label']
print(np.shape(image_tf2))
print(np.shape(label_tf2))

# lab = transfTest(label_file)
# lab_arr = np.array(lab)
# print(np.shape(lab_arr))
show_array(image_tf2[0])
show_array(label_tf2[0])



# plt.imshow(img[0,sliceIndex,:,:]) # channel, depth, height, width


In [2]:
import torch
from monai.networks.nets import UNet

# Define the UNet architecture
model_unet = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=[16, 32, 64, 128],
    strides=[(1, 2, 2), (1, 2, 2), (1, 1, 1)],
    num_res_units=2,
)

# Create an example input tensor
input_tensor = torch.randn(1, 1, 3, 192, 192)

# Pass the input tensor through the UNet model
output_tensor = model_unet(input_tensor)

# Print the size of the output tensor
print(output_tensor.size())  # Output: torch.Size([1, 2, 3, 192, 192])


torch.Size([1, 2, 3, 192, 192])
